In [1]:
# importing libraries
import pandas as pd
import sqlalchemy as db

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from api_keys import *

In [2]:
# connecting to SQL database
engine = db.create_engine(f"postgresql+psycopg2://{SQL_USER}:{SQL_PASS}@{SQL_HOST}/{SQL_DB}")
connection = engine.connect()

In [3]:
# extracting data from database
lyrics_scores = pd.read_sql_query('select * from lyrics', connection)

lyrics_scores

,index,title,album,year,lyrics,image
0,0,"​thank u, next","thank u, next",2018-11-03,Thought I'd end up with Sean\nBut he wasn't a ...,https://images.genius.com/5b04050228012fd94051...
1,1,Without Me,Manic (Target Exclusive),2018-10-04,Found you when your heart was broke\nI filled ...,https://images.genius.com/3ba32b2a382530392682...
2,2,All I Want for Christmas Is You,Merry Christmas,1994-11-01,I don't want a lot for Christmas\nThere is jus...,https://images.genius.com/c34735d6253d3e85a386...
3,3,SICKO MODE,ASTROWORLD,2018-08-03,"Astro, yeah\nSun is down, freezin' cold\nThat'...",https://images.genius.com/9c8508d3056b146aee2a...
4,4,VIRTUAL REALITY REVOLUTIONIZES CONCERTS: NEXT ...,None,2019-10-07,"Author: Jorge Ramos\nYou did read it, and it w...",https://images.genius.com/d751211ca0e16eae2d8b...
...,...,...,...,...,...,...
446,446,Slide,HER3*,2019-09-27,You always wearin' them glasses\nYou don't wan...,https://images.genius.com/a1df7845d51d3305c9e0...
447,447,October 2019 Singles Release Calendar,2019 Singles Release Calendar,None,"10/1\nAnt Clemons - ""4 Letter Word"" ft. Timbal...",https://images.genius.com/cd5f415d2040c8939d0e...
448,448,Tip Toe,Please Excuse Me for Being Antisocial,2019-11-25,"Rose gold Rollie, I'm the nigga put the whole ...",https://images.genius.com/e40bbb5187632692795d...
449,449,BEST ON EARTH,SHAKE THE SNOW GLOBE,2019-10-18,"Like the way she work, ayy\nShe always puts me...",https://images.genius.com/9c15820cb1809f83b342...


In [13]:
# adding columns to the dataframe
lyrics_scores["neg"] = 0.0
lyrics_scores["neu"] = 0.0
lyrics_scores["pos"] = 0.0
lyrics_scores["compound"] = 0.0

lyrics_scores

,index,title,album,year,lyrics,image,neg,neu,pos,compound
0,0,"​thank u, next","thank u, next",2018-11-03,Thought I'd end up with Sean\nBut he wasn't a ...,https://images.genius.com/5b04050228012fd94051...,0.0,0.0,0.0,0.0
1,1,Without Me,Manic (Target Exclusive),2018-10-04,Found you when your heart was broke\nI filled ...,https://images.genius.com/3ba32b2a382530392682...,0.0,0.0,0.0,0.0
2,2,All I Want for Christmas Is You,Merry Christmas,1994-11-01,I don't want a lot for Christmas\nThere is jus...,https://images.genius.com/c34735d6253d3e85a386...,0.0,0.0,0.0,0.0
3,3,SICKO MODE,ASTROWORLD,2018-08-03,"Astro, yeah\nSun is down, freezin' cold\nThat'...",https://images.genius.com/9c8508d3056b146aee2a...,0.0,0.0,0.0,0.0
4,4,VIRTUAL REALITY REVOLUTIONIZES CONCERTS: NEXT ...,None,2019-10-07,"Author: Jorge Ramos\nYou did read it, and it w...",https://images.genius.com/d751211ca0e16eae2d8b...,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
446,446,Slide,HER3*,2019-09-27,You always wearin' them glasses\nYou don't wan...,https://images.genius.com/a1df7845d51d3305c9e0...,0.0,0.0,0.0,0.0
447,447,October 2019 Singles Release Calendar,2019 Singles Release Calendar,None,"10/1\nAnt Clemons - ""4 Letter Word"" ft. Timbal...",https://images.genius.com/cd5f415d2040c8939d0e...,0.0,0.0,0.0,0.0
448,448,Tip Toe,Please Excuse Me for Being Antisocial,2019-11-25,"Rose gold Rollie, I'm the nigga put the whole ...",https://images.genius.com/e40bbb5187632692795d...,0.0,0.0,0.0,0.0
449,449,BEST ON EARTH,SHAKE THE SNOW GLOBE,2019-10-18,"Like the way she work, ayy\nShe always puts me...",https://images.genius.com/9c15820cb1809f83b342...,0.0,0.0,0.0,0.0


In [17]:
# list of stopwords from nltk
stop_words = set(stopwords.words("english"))

# initializing lemmatizer
lem = WordNetLemmatizer()

# initializing sentiment analyzer
sid = SentimentIntensityAnalyzer()

In [18]:
# text-processing functions

# getting part of speech of a word
# https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# lemmatizing words - reducing words to their base word
def lemmatize_words(tokenized_words: list) -> list:
    return [ lem.lemmatize(w, get_wordnet_pos(w)) for w in tokenized_words if w not in stop_words ]

def text_scores(lyrics):
    if not lyrics:
        return (0, 0, 0, 0)
    
    #tokenizing words
    tokenized_words = word_tokenize(lyrics)
    
    # lemmatizing words
    lemmatized_words = lemmatize_words(tokenized_words)

    # combining list of words back into a single string
    words_text = ' '.join(lemmatized_words)

    # scoring the lyrics
    text_scores = sid.polarity_scores(words_text)

    return text_scores.values()

In [27]:
text_scores(lyrics_scores.lyrics[0])

{'neg': 0.078, 'neu': 0.458, 'pos': 0.463, 'compound': 0.9994}

In [7]:
# iterating through the dataframe and calculating scores for the lyrics of each song

for index, row in lyrics_scores.iterrows():
    lyrics_scores.loc[lyrics_scores['index'] == index, ['neg', 'neu', 'pos', 'compound']] = (0, 0, 0, 0)#text_scores(row.lyrics)

Thought I'd end up with Sean
But he wasn't a match
Wrote some songs about Ricky
Now I listen and laugh
Even almost got married
And for Pete, I'm so thankful
Wish I could say, "Thank you" to Malcolm
'Cause he was an angel

One taught me love
One taught me patience
And one taught me pain
Now, I'm so amazing
Say I've loved and I've lost
But that's not what I see
So, look what I got
Look what you taught me
And for that, I say

Thank you, next (Next)
Thank you, next (Next)
Thank you, next
I'm so fuckin' grateful for my ex
Thank you, next (Next)
Thank you, next (Next)
Thank you, next (Next)
I'm so fuckin'—

Spend more time with my friends
I ain't worried 'bout nothin'
Plus, I met someone else
We havin' better discussions
I know they say I move on too fast
But this one gon' last
'Cause her name is Ari
And I'm so good with that (So good with that)

She taught me love (Love)
She taught me patience (Patience)
How she handles pain (Pain)
That shit's amazing (Yeah, she's amazing)
I've loved and I'

With arsenic sauce
Walk in your rainbow paradise (Paradise)
Strawberry lipstick state of mind (State of mind)
I get so lost inside your eyes
Would you believe it?

You don't have to say you love me
You don't have to say nothing
You don't have to say you're mine

Honey (Ah-ah-ah)
I'd walk through fire for you
Just let me adore you
Oh, honey (Ah-ah-ah)
I'd walk through fire for you
Just let me adore you
Like it's the only thing I'll ever do
Like it's the only thing I'll ever do

You're wonder under summer sky (Summer sky)
Brown skin and lemon over ice
Would you believe it?

You don't have to say you love me
I just wanna tell you somethin'
Lately, you've been on my mind

Honey (Ah-ah-ah)
I'd walk through fire for you
Just let me adore you
Oh, honey (Ah-ah-ah)
I'd walk through fire for you
Just let me adore you
Like it's the only thing I'll ever do
Like it's the only thing I'll ever do

It's the only thing I'll ever do
It's the only thing I'll ever do
It's the only thing I'll ever do
It's 

In [168]:
lyrics_scores

,index,title,album,year,lyrics,image,neg,neu,pos,compound
0,0,"​thank u, next","thank u, next",2018-11-03,Thought I'd end up with Sean\nBut he wasn't a ...,https://images.genius.com/5b04050228012fd94051...,0.078,0.458,0.463,0.9994
1,1,Without Me,Manic (Target Exclusive),2018-10-04,Found you when your heart was broke\nI filled ...,https://images.genius.com/3ba32b2a382530392682...,0.127,0.799,0.074,-0.8355
2,2,All I Want for Christmas Is You,Merry Christmas,1994-11-01,I don't want a lot for Christmas\nThere is jus...,https://images.genius.com/c34735d6253d3e85a386...,0.121,0.744,0.136,0.9594
3,3,SICKO MODE,ASTROWORLD,2018-08-03,"Astro, yeah\nSun is down, freezin' cold\nThat'...",https://images.genius.com/9c8508d3056b146aee2a...,0.107,0.596,0.298,0.9986
4,4,VIRTUAL REALITY REVOLUTIONIZES CONCERTS: NEXT ...,None,2019-10-07,"Author: Jorge Ramos\nYou did read it, and it w...",https://images.genius.com/d751211ca0e16eae2d8b...,0.041,0.781,0.178,0.9939
...,...,...,...,...,...,...,...,...,...,...
446,446,Slide,HER3*,2019-09-27,You always wearin' them glasses\nYou don't wan...,https://images.genius.com/a1df7845d51d3305c9e0...,0.078,0.835,0.087,-0.3891
447,447,October 2019 Singles Release Calendar,2019 Singles Release Calendar,None,"10/1\nAnt Clemons - ""4 Letter Word"" ft. Timbal...",https://images.genius.com/cd5f415d2040c8939d0e...,0.088,0.821,0.091,0.8711
448,448,Tip Toe,Please Excuse Me for Being Antisocial,2019-11-25,"Rose gold Rollie, I'm the nigga put the whole ...",https://images.genius.com/e40bbb5187632692795d...,0.213,0.715,0.072,-0.9969
449,449,BEST ON EARTH,SHAKE THE SNOW GLOBE,2019-10-18,"Like the way she work, ayy\nShe always puts me...",https://images.genius.com/9c15820cb1809f83b342...,0.132,0.646,0.222,0.9825


In [169]:
# sending table to SQL
lyrics_scores.to_sql("lyrics_scores", connection, if_exists = "replace")

connection.close()